## 1. Install libraries

In [ ]:
pip install pandas numpy matplotlib seaborn scipy scikit-learn xgboost

## 2.  Import & Load Data

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD


# Load the data from the csv file and display the first 5 rows
path = '/Users/danielmashala/dev/code/ds_code/Maccabi_Home_Task/ds_assignment_data.csv'
df = pd.read_csv(path)

# Display the first 5 rows of the data
df.head(5)

## 3. Overview of Columns
Identify feature groups and key columns.

In [ ]:
# Dataset shape
print(f"Dataset shape: {df.shape[0]} rows, {df.shape[1]} columns\n")
# Dataset data types
dtype_counts = df.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"Number of columns with dtype {dtype}: {count}")
print("\n")

# Overview of Columns
demographic_cols = [col for col in df.columns if col.startswith('demog_')]
lab_cols = [col for col in df.columns if col.startswith('lab_')]
bp_cols = [col for col in df.columns if col.startswith('measure_blood_pressure')]
diagnosis_cols = [col for col in df.columns if 'diag' in col and 'match' not in col]
match_cols = [col for col in df.columns if col.startswith('match_')]
subtype_cols = [col for col in df.columns if col.endswith('_sum')]
clinical_text_col = 'clinical_sheet'
target_col = 'Y'
print(f'Demographics: {demographic_cols}')
print(f'Lab: {lab_cols[:5]} ...')
print(f'BP: {bp_cols[:5]} ...')
print(f'Diagnosis: {diagnosis_cols[:5]} ...')
print(f'Match flags: {match_cols}')
print(f'Subtype counts: {subtype_cols}')
print(f'Clinical text: {clinical_text_col}')
print(f'Target: {target_col}')

## 4. Missingness Analysis
Visualize and summarize missing data patterns.

In [ ]:
missing = df.isnull().mean().sort_values(ascending=False)
plt.figure(figsize=(12,4))
sns.histplot(missing, bins=30, kde=False)
plt.title('Distribution of Missingness Across Features')
plt.xlabel('Fraction Missing')
plt.ylabel('Number of Features')
plt.show()
print('Top 20 features with most missing values:')
print(missing.head(20))

In [ ]:
# Calculate missing value fractions for each column
missing_fraction = df.isnull().mean()

# 1. Number of columns with >80% missing values
num_cols_over_80 = (missing_fraction >= 0.8).sum()

# 2. Number of columns with no missing values
num_cols_no_missing = (missing_fraction == 0).sum()

# 3. Number of columns between 0 and 50% missing values
num_cols_under_50_over_0 = ((missing_fraction < 0.5) & (missing_fraction > 0)).sum()

# 4. Number of columns between 50% and 80% missing values
num_cols_over_50_under_80 = ((missing_fraction > 0.5) & (missing_fraction < 0.8)).sum()

# 5. Number of columns with >90% missing values
num_cols_over_90 = (missing_fraction >= 0.9).sum()

print(f"Columns with no missing values: {num_cols_no_missing}")
print(f"Columns with 0<cols<50% missing values: {num_cols_under_50_over_0}")
print(f"Columns with 50%<cols<80% missing values: {num_cols_over_50_under_80}")
print(f"Columns with >= 80% missing values: {num_cols_over_80}")
print(f"Columns with >= 90% missing values: {num_cols_over_90}")

## 5. Target 'Y' Distribution 
Examine the balance of the target variable.

In [ ]:
y_counts = df['Y'].value_counts()
y_percentage = df['Y'].value_counts(normalize=True) * 100

plt.figure(figsize=(4,3))
sns.countplot(x='Y', data=df)
plt.title('Target (Y) Distribution')
plt.show()

print("Target (Y) Distribution:")
print("Counts:")
print(y_counts)
print("Percentage:")
print(y_percentage)

#### Target Variable (Y) – Hypertensive Complications in Pregnancy
0 – No complication: 9,568 cases (95.68%)

1 – Complication (e.g., preeclampsia, gestational hypertension): 432 cases (4.32%)


Insights:

This is an imbalanced classification problem – only ~4% positive cases.

Focus on metrics beyond accuracy:

Recall – catch as many high-risk cases as possible

Precision at top-k – to prioritize who should be referred for further examination

F1 score – better suited for imbalanced data


#### Distribution of Source Flags

In [ ]:
flag_cols = ["match_diag_after", "match_measure_after",
             "match_rasham_after", "match_aspirin_after", "match_pdf_after"]

y_flags = df[df["Y"] == 1][flag_cols].sum()
y_flags_pct = y_flags / y_flags.sum() * 100

plt.figure(figsize=(7, 7))
plt.pie(y_flags_pct, labels=y_flags_pct.index, autopct="%1.1f%%")
plt.title("Distribution of Diagnosis Sources (Y = 1)")
plt.tight_layout()
plt.show()
print(y_flags_pct.sort_values(ascending=False))


#### Summary: Source of Positive Cases (Y = 1)

Among the 432 positive cases (Y = 1), the distribution of source flags indicates:

- **54% (`match_diag_after`)** were identified via ICD-9 diagnosis codes — the main administrative channel.
- **22% (`match_aspirin_after`)** had a low-dose aspirin prescription, typically reflecting clinical suspicion of risk.
- **21% (`match_pdf_after`)** were identified from hospital admission/discharge notes — likely more severe cases.
- **3% (`match_rasham_after`)** came from general medical registry ("Rasham"), suggesting limited documentation via that channel.
- **0% (`match_measure_after`)** were flagged solely via blood pressure measurements, indicating BP data is not actively used in current labeling.

**Conclusion:** The label (Y) heavily depends on post-week-15 diagnosis codes and clinical documentation. Therefore, early signals such as blood pressure, urine protein, or early lab results are likely underrepresented in Y = 1. This should be taken into account when training a model for early prediction.


#### Sub-Complication Breakdown (Among the 432 Cases Where Y = 1)

In [ ]:
subtype_columns = [
    'secondary_hypertension_sum',
    'essential_hypertension_sum',
    'hypertensive_heart_disease_sum',
    'hypertensive_chronic_kidney_disease_sum',
    'hypertensive_heart_and_chronic_kidney_disease_sum',
    'pregnancy_hypertension_sum',
    'preeclampsia_sum',
    'eclampsia_sum'
]

# Count how many cases of each type occurred (at least one occurrence)
subtype_counts = (df[subtype_columns] > 0).sum().sort_values(ascending=False)

subtype_counts

#### Detected sub-complications:

- preeclampsia_sum: 128 cases – Preeclampsia, the most common and dangerous complication

- pregnancy_hypertension_sum: 111 cases – Gestational hypertension without organ damage

- essential_hypertension_sum: 100 cases – Primary hypertension that developed during pregnancy

- eclampsia_sum: 17 cases – Severe form of preeclampsia with seizures

Sub-complications that did not appear at all:

- secondary_hypertension_sum

- hypertensive_heart_disease_sum

- hypertensive_chronic_kidney_disease_sum

- hypertensive_heart_and_chronic_kidney_disease_sum

These may be irrelevant for pregnancy or simply not recorded in the dataset.

Conclusions:

- Preeclampsia is the most common and critical complication to detect.
- Eclampsia is rare but very severe – worth considering a model sensitive to extreme cases.
- The Y column aggregates all types, but training a separate model for a specific complication (e.g., preeclampsia) might also be valuable depending on the use case.

## Age Distribution
Visualize age distribution by target.

In [ ]:
# remove missing age values if exisit 
df = pd.read_csv(path)
df = df[df["demog_customer_age"].notna()].copy()

# Set the age column name (adjust if your column is named differently)
age_col = 'demog_customer_age'

# Plot age distribution by Y
plt.figure(figsize=(8,5))
sns.histplot(data=df, x=age_col, hue='Y', bins=30, kde=True, stat='density', common_norm=False)
plt.title('Age Distribution by Target (Y)')
plt.xlabel('Age')
plt.ylabel('Density')
plt.legend(title='Y', labels=['No Complication (0)', 'Complication (1)'])
plt.show()


age_stats = df.groupby('Y')[age_col].agg(['count', 'mean', 'std', 'min', 'max', 'median'])
print(age_stats)

In [ ]:
from scipy.stats import ttest_ind

age_0 = df[df['Y'] == 0][age_col]
age_1 = df[df['Y'] == 1][age_col]
t_stat, p_val = ttest_ind(age_0, age_1, equal_var=False)
print(f"T-test: T-stat={t_stat:.2f} p-value: {p_val:.5f}")


### Age and Risk of Hypertensive Disorders
To assess the relationship between age and the risk of hypertensive disorders, the distribution of age was compared between patients who developed complications (Y=1) and those who did not (Y=0).

Patients with complications had a slightly higher average age (30.8 years) compared to those without complications (29.7 years). The distribution plot shows a modest shift toward older age among the complication group.

t-test confirmed that this difference is statistically significant (t = -3.80, p = 0.00016).

Conclusion: While the difference in age is small, older maternal age is associated with an increased risk of hypertensive disorders and may contribute useful signal when included as a feature in predictive models.

In [ ]:
# Create binary indicators for each complication
df['has_preeclampsia'] = df['preeclampsia_sum'] > 0
df['has_pregnancy_htn'] = df['pregnancy_hypertension_sum'] > 0
df['has_eclampsia'] = df['eclampsia_sum'] > 0

# Create boxplots to compare age across complication categories
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sns.boxplot(data=df, x='has_preeclampsia', y='demog_customer_age', ax=axes[0])
axes[0].set_title("Age vs Preeclampsia")
axes[0].set_xticklabels(['No', 'Yes'])

sns.boxplot(data=df, x='has_pregnancy_htn', y='demog_customer_age', ax=axes[1])
axes[1].set_title("Age vs Pregnancy Hypertension")
axes[1].set_xticklabels(['No', 'Yes'])

sns.boxplot(data=df, x='has_eclampsia', y='demog_customer_age', ax=axes[2])
axes[2].set_title("Age vs Eclampsia")
axes[2].set_xticklabels(['No', 'Yes'])

plt.tight_layout()
plt.show()

This plot compares age distributions across patients with and without three hypertensive disorder subtypes.

Preeclampsia and Pregnancy Hypertension show similar age distributions between affected and unaffected groups, suggesting age is not a strong predictor for these conditions.

Eclampsia shows a noticeable shift toward older ages in affected patients, indicating age may play a more important role.

Conclusion: Age appears to be more relevant for predicting eclampsia than for other hypertensive disorder subtypes.

#### Demog_capitationcoefficient Distribution:

In [ ]:
df = pd.read_csv(path)

# Plot distribution of demog_capitationcoefficient by target
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x="Y", y="demog_capitationcoefficient")
plt.title("Distribution of Capitation Coefficient by Target (Y)")
plt.xlabel("Target (0 = No , 1 = Hypertensive)")
plt.ylabel("Capitation Coefficient (Socioeconomic Proxy)")
plt.xticks([0, 1], ['No', 'Hypertensive'])
plt.grid(axis='y')
plt.tight_layout()
plt.show()


# Group by target and calculate mean and standard deviation of capitation coefficient
summary_stats = df.groupby("Y")["demog_capitationcoefficient"].agg(["mean", "std", "median", "count"])
summary_stats

In [ ]:
from scipy.stats import ttest_ind

group_0 = df[df['Y'] == 0]['demog_capitationcoefficient'].dropna()
group_1 = df[df['Y'] == 1]['demog_capitationcoefficient'].dropna()

# T-test
t_stat, p_value = ttest_ind(group_0, group_1, equal_var=False)
print(f"T-test: T-stat={t_stat:.2f} p-value: {p_value:.5f}")

### Socioeconomic Status and Hypertensive Disorders
To evaluate the relationship between socioeconomic status (proxied by demog_capitationcoefficient) and hypertensive disorders:

- The mean capitation coefficient was slightly higher among patients who developed HDP (Y=1: 0.742) compared to those who did not (Y=0: 0.720).

- Both groups had the same median (0.74), but variance was higher in the HDP group.

A t-test confirmed that the difference in means is statistically significant:

t = -3.83, p = 0.00015

Conclusion:

Socioeconomic status shows a weak but statistically significant association with hypertensive outcomes. While not a strong standalone predictor, it may contribute useful signal when combined with clinical features.

## 8. Correlation with Target
Show features most correlated with the target.

#### Data Description:


- The prediction is made at Week 15 of gestation so only features available up to that point can be used as model inputs to avoid data leakage.

- The cohort includes only low and moderate risk pregnancies. High-risk patients are excluded as they are already referred for testing.

- The target variable Y indicates whether a hypertensive complication (e.g., preeclampsia, gestational hypertension, or eclampsia) occurred later in pregnancy.

- Source flags (e.g., diagnosis codes, aspirin prescription) indicate how the outcome was identified and are useful for label validation, NOT for prediction. These features are derived after Week 15 and must not be used as predictors to avoid data leakage.

- An initial scan of the dataset reveals that 45 columns have over 80% missing values. These features are unlikely to contribute meaningful signals and may introduce noise or instability in the model.

- Diagnostic subtype columns (e.g., preeclampsia_sum, pregnancy_hypertension_sum, eclampsia_sum) are also excluded from modeling and correlation analysis. These variables are generated based on post-week-15 diagnosis data and therefore reflect the outcome itself. Including them would result in data leakage and overestimation of predictive performance.

In [ ]:
# Load the data 
df = pd.read_csv(path)

#remove source flags and subtype columns
numeric_cols = [
    col for col in df.select_dtypes(include=[np.number]).columns # Select numeric columns only
    if not col.startswith('match_') and not col.endswith('_sum')
]

# Compute full correlation matrix (pairwise deletion)
corr_matrix = df[numeric_cols].corr(method="pearson")

# Build an n_pairs matrix: number of non‑NaN pairs for every column pair
non_na = df[numeric_cols].notna().astype(int)
n_pairs_matrix = non_na.T.dot(non_na)    # dot‑product counts pairwise valid rows

# Extract correlation & n_pairs versus the target Y
corr_vs_y = corr_matrix["Y"].drop("Y")           # remove self‑correlation
n_pairs_vs_y = n_pairs_matrix["Y"].drop("Y")

result = (
    pd.DataFrame({
        "corr": corr_vs_y,
        "abs_corr": corr_vs_y.abs(),
        "n_pairs": n_pairs_vs_y
    })
    .sort_values("abs_corr", ascending=False)
)

# Show the full table or just the top‑ 20
print("=== Correlation vs. Y (with n_pairs) – top 20 ===\n")
print(result.head(20).to_string())

#### Top correlated features with Y include:

- Diagnosis recency features like 24_diag_53_days_since_last_diag (r = 0.34) and 4_diag_98_days_since_last_diag (r = 0.31)

- Blood pressure stats (e.g., systolic/diastolic mean, max, last) show consistent moderate correlation (~0.12–0.14)

- Weight at lab time also shows a mild positive signal (r = 0.11)

⚠️ Some diagnosis features have few samples (n_pairs < 30), but due to their strong signal, I keep them for now and will handle sparsity via feature engineering later.

### Missing Values: 

#### Age: 

In [13]:
# Define a clean function to only fill age where missing, using text
def fill_age_from_text(df):
    def extract_age_from_text(text):
        match = re.search(r'\b(?:|בת|גיל)\s{0,3}(\d{2})\b', str(text))
        if match:
            age = int(match.group(1))
            if 15 <= age <= 60:
                return age
        return None

    # Extract candidate age from text
    df['age_from_text'] = df['clinical_sheet'].apply(extract_age_from_text)

    # Fill only where missing
    df.loc[df['demog_customer_age'].isna(), 'demog_customer_age'] = df.loc[
        df['demog_customer_age'].isna(), 'age_from_text'
    ]

    return df

df = pd.read_csv(path)
missing_age_before =  df[df['demog_customer_age'].isna()]
# Apply function to fill missing age from clinical text
df_after = fill_age_from_text(df)

missing_age_after = df_after[df_after['demog_customer_age'].isna()]



In [ ]:
print("Befor:" ,missing_age_before['demog_customer_age'])
print("After: ",missing_age_after['demog_customer_age'])    

#### Removing Patients with Multiple Missing Demographics
After analyzing the 9 patients with missing demog_customer_age, I observed that these same patients were also missing critical demographic variables:

demog_capitationcoefficient

All smoking-related features (smoking_is_smoker, smoking_smoking_years, etc.)

Since these records lacked multiple key inputs and none of them were positive cases (Y=1), I chose to remove them entirely from the dataset to maintain data quality and avoid noisy imputation.

This left a final dataset with 9,991 valid patient records.

### Smoking cols:

In [ ]:

df = pd.read_csv(path)

# Check if the smoking related columns have missing values in the same rows
smoking_cols = ['smoking_is_smoker', 'smoking_smoking_years', 'smoking_total_heavy_smokers']
smoking_na_matrix = df[smoking_cols].isna()

# Check how many rows have all 3 missing
all_missing = smoking_na_matrix.all(axis=1).sum()

# Check how many rows have partial missing (some but not all)
partial_missing = smoking_na_matrix.any(axis=1).sum() - all_missing

all_missing, partial_missing

#### Handling Missing Smoking Data
There are 3,269 patients (nearly one third of the dataset) with missing values for all three smoking related features.
Rather than removing these rows, I chose to retain them and apply a targeted imputation strategy:

smoking_is_smoker: Filled from text where possible. If no information was found, defaulted to 0 (non-smoker), with a missing indicator added.

smoking_smoking_years and smoking_total_heavy_smokers: Filled with 0 only when the patient was inferred as a non-smoker. Missingness was captured via indicator columns.

This approach preserves valuable data and allows the model to handle uncertainty effectively.

### Lab Data: 

In [ ]:
# Reload dataset
df = pd.read_csv(path)

# Count missing values for all lab_ columns
lab_missing_summary = df.filter(like='lab_').isna().sum().sort_values(ascending=False)

lab_missing_summary

### Handling Missing Values in Lab Features
Most lab_ features had very few missing values (between 1–74 records), which is negligible.
These were filled using the median, with corresponding missing indicator columns added.

The lab_Protein-U_last_value feature had a higher missing rate (~42%).
For simplicity, I also filled it with the median and added a missing indicator.
In a full production setting, I would consider exploring the missingness mechanism or use predictive imputation techniques for this variable.

### Measure Blood Pressure data:

In [ ]:
# find missing values in cols that start with measure_blood_pressure
bp_cols = [col for col in df.columns if col.startswith('measure_blood_pressure')]
bp_missing_summary = df[bp_cols].isna().sum().sort_values(ascending=False)

# Check if the missing data is on the same rows 
bp_missing_matrix = df[bp_cols].isna()
n_rows_with_all_missing = bp_missing_matrix.all(axis=1).sum()
n_rows_with_partial_missing = bp_missing_matrix.any(axis=1).sum() - n_rows_with_all_missing

bp_missing_summary

### Handling Missing Values in Blood Pressure Features
About 50% of the patients were missing all blood pressure features.
Since this likely reflects a clinical decision not to measure BP in low-risk women, I filled all missing values with -1 and added a binary indicator (has_bp_data) to capture the presence of any BP measurements.
This lets the model distinguish between low values and true missingness.

In [ ]:
def preprocess_missing_values(df):
    # 1. Labs: fill with median and create _missing indicators
    lab_cols = [col for col in df.columns if col.startswith('lab_')]
    for col in lab_cols:
        if df[col].isna().any():
            df[f'{col}_missing'] = df[col].isna().astype(int)
            df[col] = df[col].fillna(df[col].median())

    # 2. Free-text field: clinical notes
    df['clinical_sheet'] = df['clinical_sheet'].fillna('')

    # 3. Capitation coefficient - only 9 values are missing and is the same with the age raw will remove. 
    # to be sure the rest if have na value complete it with meadian
    if 'demog_capitationcoefficient' in df.columns:
        df['demog_capitationcoefficient_missing'] = df['demog_capitationcoefficient'].isna().astype(int)
        df['demog_capitationcoefficient'] = df['demog_capitationcoefficient'].fillna(
            df['demog_capitationcoefficient'].median()
        )

    # 4. Customer age — drop rows where it was missing - only 9 is missing
    df = df[df['demog_customer_age'].notna()].reset_index(drop=True)

    # 5. Smoking
    # Fill smoking_is_smoker using text_says_smoker if available
    if 'text_says_smoker' in df.columns and 'smoking_is_smoker' in df.columns:
        mask = df['smoking_is_smoker'].isna() & df['text_says_smoker'].notna()
        df.loc[mask, 'smoking_is_smoker'] = df.loc[mask, 'text_says_smoker']
    
    # Fill the rest of smoking_is_smoker with 0 (default = non-smoker)
    df['smoking_is_smoker_missing'] = df['smoking_is_smoker'].isna().astype(int)
    df['smoking_is_smoker'] = df['smoking_is_smoker'].fillna(0)

    # Now conditionally fill other smoking features
    # smoking_smoking_years
    if 'smoking_smoking_years' in df.columns:
        df['smoking_smoking_years_missing'] = df['smoking_smoking_years'].isna().astype(int)
        df.loc[(df['smoking_smoking_years'].isna()) & (df['smoking_is_smoker'] == 0), 'smoking_smoking_years'] = 0

    # smoking_total_heavy_smokers
    if 'smoking_total_heavy_smokers' in df.columns:
        df['smoking_total_heavy_smokers_missing'] = df['smoking_total_heavy_smokers'].isna().astype(int)
        df.loc[(df['smoking_total_heavy_smokers'].isna()) & (df['smoking_is_smoker'] == 0), 'smoking_total_heavy_smokers'] = 0
    
    # 6. Blood Pressure: create has_bp_data indicator, fill missing with -1
    bp_cols = [col for col in df.columns if col.startswith('measure_blood_pressure')]
    df['has_bp_data'] = (~df[bp_cols].isna().all(axis=1)).astype(int)
    df[bp_cols] = df[bp_cols].fillna(-1)
    
    # 7. fill diagnosis columns - Fill NaNs with 999 to indicate "no diagnosis"
    diag_4m_cols = [col for col in df.columns if col.startswith('4_diag_') and 'days_since_last_diag' in col]
    diag_24m_cols = [col for col in df.columns if col.startswith('24_diag_') and 'days_since_last_diag' in col]
    df[diag_4m_cols] = df[diag_4m_cols].fillna(999)
    df[diag_24m_cols] = df[diag_24m_cols].fillna(999)

    return df

df = pd.read_csv(path)
print(df.shape)
df_missing = preprocess_missing_values(df)
df_missing.head(10)


In [ ]:
# Check for NaN values 
nan_check = df_missing.isna().sum()
print(nan_check[nan_check > 0])

#### Diagnosis History Strategy:
Raw *_days_since_last_diag columns are excluded from the model
Instead, I engineered aggregate features:
 - num_recent_diags_4m / 24m
 - min_days_since_diag_4m / 24m
 - recency_score_4m / 24m
 
These retain the clinical signal in a compressed, generalizable form

In [ ]:

def generate_diagnosis_features(df):
    # Identify relevant diagnosis columns
    diag_4m_cols = [col for col in df.columns if col.startswith('4_diag_') and 'days_since_last_diag' in col]
    diag_24m_cols = [col for col in df.columns if col.startswith('24_diag_') and 'days_since_last_diag' in col]

    # Fill NaNs with 999 to indicate "no diagnosis"
    df[diag_4m_cols] = df[diag_4m_cols].fillna(999)
    df[diag_24m_cols] = df[diag_24m_cols].fillna(999)

    # Create binary flags: whether diagnosis exists in time window
    diag_4m_flags = df[diag_4m_cols] < 999
    diag_24m_flags = df[diag_24m_cols] < 999

    # Aggregated features
    df['num_recent_diags_4m'] = diag_4m_flags.sum(axis=1)
    df['num_recent_diags_24m'] = diag_24m_flags.sum(axis=1)
    df['min_days_since_diag_4m'] = df[diag_4m_cols].min(axis=1).clip(upper=150)
    df['min_days_since_diag_24m'] = df[diag_24m_cols].min(axis=1).clip(upper=730)
    df['recency_score_4m'] = np.exp(-df['min_days_since_diag_4m'] / 30)
    df['recency_score_24m'] = np.exp(-df['min_days_since_diag_24m'] / 90)
    
    return df

# Apply to your dataframe
diagnosis_df = generate_diagnosis_features(df)
diagnosis_df.head()


### Extracting Clinical Text Features:
The free-text clinical notes (clinical_sheet) were processed using the following steps:

Text Cleaning
Removed irrelevant numeric entities (e.g., weights, ages, years) and punctuation to reduce noise in the text.

TF-IDF Vectorization
Transformed the cleaned text into numerical features using Term Frequency–Inverse Document Frequency (TF-IDF) with a limit of 500 features.

Dimensionality Reduction (SVD)
Applied Truncated SVD to reduce the TF-IDF features to 50 components (txt_svd_0 to txt_svd_49), capturing the main semantic signals while minimizing dimensionality.

In [ ]:
# Clinical Text Features

def extract_text_features(df, text_column='clinical_sheet', max_features=500, svd_components=50):
    def clean_clinical_text(text):
        text = str(text).lower()
        text = re.sub(r'\d{2,3}\s?(kg|cm|years?|שנים|ק"ג|קג)', '', text)
        text = re.sub(r'\d{4}', '', text)  # remove years like 1992, 2021
        text = re.sub(r'\n|\r', ' ', text)
        text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
        return text.strip()

    # Step 1: Clean the text
    df['clinical_text_clean'] = df[text_column].fillna('').map(clean_clinical_text)

    # Step 2: TF-IDF Vectorization
    tfidf = TfidfVectorizer(max_features=max_features)
    X_tfidf = tfidf.fit_transform(df['clinical_text_clean'])

    # Step 3: Dimensionality Reduction
    svd = TruncatedSVD(n_components=svd_components, random_state=42)
    X_text_features = svd.fit_transform(X_tfidf)

    # Step 4: Merge with original DataFrame
    tfidf_feature_names = [f'txt_svd_{i}' for i in range(X_text_features.shape[1])]
    df_tfidf = pd.DataFrame(X_text_features, columns=tfidf_feature_names, index=df.index)

    return df_tfidf

# Apply the function now
df = extract_text_features(df)
df



In [22]:
# Define full feature engineering pipeline
def full_feature_engineering(df):
    # ==== 1. Blood Pressure Features ====
    df['bp_sys_range'] = df['measure_blood_pressure_sys_max_val'] - df['measure_blood_pressure_sys_min_val']
    df['bp_sys_trend'] = df['measure_blood_pressure_sys_last_val'] - df['measure_blood_pressure_sys_first_val']
    df['bp_high_flag'] = ((df['measure_blood_pressure_sys_mean_val'] > 130) | 
                          (df['measure_blood_pressure_dias_mean_val'] > 85)).astype(int)
    df['bp_x_age'] = df['measure_blood_pressure_sys_mean_val'] * df['demog_customer_age']

    # ==== 2. Lab-Based Features ====
    df['neutro_to_lymph_ratio'] = df['lab_Neutrophils_1_last_value'] / (df['lab_Lymphocytes_1_last_value'] + 1e-5)
    df['hgb_low_flag'] = (df['lab_Hemoglobin (HGB)_last_value'] < 11).astype(int)

    # ==== 3. Text Feature ====
    df['txt_length'] = df['clinical_sheet'].fillna('').apply(lambda x: len(str(x).split()))

    return df

df = pd.read_csv(path)
df = full_feature_engineering(df)


### Data Pipline:

In [ ]:
# Reload dataset
df = pd.read_csv(path)

# ==== Step 1: Preprocess missing values ====
df = preprocess_missing_values(df)

# ==== Step 2: Generate text features ====
df_text_features = extract_text_features(df)
df = pd.concat([df, df_text_features], axis=1)

# ==== Step 3: Feature engineering ====
df = generate_diagnosis_features(df)
df = full_feature_engineering(df)

# ==== Step 4: Remove subtype and diagnosis col start with 4/24_diag ====
df = df.drop(columns=[col for col in df.columns 
                     if (col.startswith('match_') and not col.endswith('_sum')) or
                        (col.startswith('4_diag_') and 'days_since_last_diag' in col) or
                        (col.startswith('24_diag_') and 'days_since_last_diag' in col) or
                        col.endswith('_sum')])

# ==== Step 5: Feature selection ====
exclude_cols = [
    'Y', 'clinical_sheet', 'clinical_text_clean', 'text_says_smoker', 'int_date'
]
X = df[[col for col in df.columns if col not in exclude_cols]]
y = df['Y']

print("X shape:", X.shape)
print("y shape:", y.shape)

In [ ]:
# Create one combined DataFrame
df_combined = X.copy()
df_combined['target'] = y

# Export to CSV
df_combined.to_csv('/Users/danielmashala/dev/code/ds_code/Maccabi_Home_Task/X_y_combined.csv', index=False)

print(f"Combined table exported with shape: {df_combined.shape}")
print(f"Features: {len(df_combined.columns) - 1}")  # -1 for target column
print(f"Target column: 'target'")

### Trainning the Model

In [36]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Train/test split - use stratify to balance the y label in train and test datasets 
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


In [64]:

# Calculate scale_pos_weight to handle class imbalance in XGBoost
# Formula: scale_pos_weight = (Number of Negative Samples) / (Number of Positive Samples)
neg_count = (y == 0).sum()
pos_count = (y == 1).sum()
scale_pos_weight = neg_count / pos_count


# Train models
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)
rf_proba = rf_model.predict_proba(X_test)[:, 1]

xgb_model = XGBClassifier(n_estimators=100, eval_metric='logloss', random_state=42, scale_pos_weight=scale_pos_weight)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)
xgb_proba = xgb_model.predict_proba(X_test)[:, 1]

# Evaluate
rf_auc = roc_auc_score(y_test, rf_proba)
xgb_auc = roc_auc_score(y_test, xgb_proba)
report_rf = pd.DataFrame(classification_report(y_test, rf_preds, output_dict=True)).transpose().round(3)
report_xgb = pd.DataFrame(classification_report(y_test, xgb_preds, output_dict=True)).transpose().round(3)
report_rf.loc['AUC'] = [rf_auc, '', '', '']
report_xgb.loc['AUC'] = [xgb_auc, '', '', '']

def plot_confusion_matrix(y_true, y_pred, model_name):
    """Plot confusion matrix for a model"""
    cm = confusion_matrix(y_true, y_pred)
    
    # Create and plot with blue colormap
    ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1]).plot(cmap=plt.cm.Blues)
    plt.title(f'Confusion Matrix - {model_name}')
    plt.show()
    
    # Print metrics
    tn, fp, fn, tp = cm.ravel()
    print(f"{model_name} Results:")
    print(f"True Negatives: {tn}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    print(f"True Positives: {tp}")
    print(f"Precision: {tp/(tp+fp):.3f}")
    print(f"Recall: {tp/(tp+fn):.3f}")
    print("---")


In [ ]:
print("Random Forest Classification Report:")
report_rf

In [ ]:
# Plot confusion matrices
plot_confusion_matrix(y_test, rf_preds, "Random Forest")

### Random Forest Model Evaluation – Confusion Matrix Summary

he confusion matrix summarizes the prediction performance of the Random Forest model trained with class_weight='balanced'.

Key Observations:
- The model predicted only 2 true positives out of 86 actual class 1 cases → recall of just ~2.3%.

- While precision for class 1 was perfect (1.0), this is misleading because the model barely predicted any positives at all.

- It achieved zero false positives, indicating extreme caution — but at the cost of missing nearly all the true cases.

- Despite a decent overall accuracy (95.8%), the model failed in the primary business goal: identifying high-risk pregnancies early.

Conclusion:
- Random Forest, even with class balancing, showed very poor recall for the minority class (patients with hypertensive conditions).

- The model appears to be overly conservative, effectively ignoring the positive class to avoid false positives.

- This behavior makes it unsuitable for clinical risk prediction where recall (sensitivity) is far more critical than overall accuracy.

In [ ]:
print("XGBoost Classification Report:")
report_xgb

In [ ]:
plot_confusion_matrix(y_test, xgb_preds, "XGBoost")

### XGBoost Model Evaluation – Confusion Matrix Summary

The confusion matrix summarizes the prediction performance of the XGBoost model, trained with class imbalance handling using scale_pos_weight.

Key Observations:
- The model correctly identified 50 out of 86 true positive cases (recall ≈ 58.1%).

- It made only 10 false positive predictions, indicating a high precision of ~83.3% for class 1.

- Overall accuracy was 97.7%, with an excellent AUC of 0.974, indicating strong discriminative power.

- The balance between sensitivity (recall) and specificity (TN rate) is particularly important in clinical risk screening, where missing true positives could result in serious complications.


Conclusion:
- XGBoost performed well on this imbalanced medical dataset, successfully identifying a substantial portion of high-risk pregnancies.

- Its high precision and reasonable recall make it a strong candidate for clinical decision support systems, especially when early detection is critical and false positives are tolerable under cost-effective interventions like aspirin administration.

### Screening Prioritization – Model-Based Ranking
An XGBoost model was trained to assign each patient a risk score indicating the likelihood of developing a hypertensive disorder. Instead of using the model for binary classification, patients were ranked by predicted risk, and performance was evaluated at various Top-K thresholds (representing different screening budget levels):

In [ ]:
# Build ranking dataframe
ranking_df = pd.DataFrame({
    'patient_id': X_test.index,
    'true_label': y_test.values,
    'xgb_risk_score': xgb_proba
}).sort_values(by='xgb_risk_score', ascending=False).reset_index(drop=True)

# Evaluate at different budget thresholds
k_values = [50, 100, 200, 300, 500]
total_positives = ranking_df['true_label'].sum()
ranking_summary = []

for k in k_values:
    top_k = ranking_df.head(k)
    recall_at_k = top_k['true_label'].sum() / total_positives
    precision_at_k = top_k['true_label'].sum() / k
    ranking_summary.append({'Top K': k, 'Precision@K': precision_at_k, 'Recall@K': recall_at_k})

ranking_metrics = pd.DataFrame(ranking_summary)
ranking_metrics


#### Observations:

- Screening the top 100–200 patients captures the majority of true positive cases while limiting the number of costly lab tests.

- This ranking approach is suitable for screening prioritization when resources are limited and early detection is critical.

- Precision declines as K increases, as expected, while recall improves — reflecting a typical trade-off in prioritization.


Model Improvement & Future Work: 
- Feature Importance Analysis
Use XGBoost gain-based importance to remove low-impact features and focus on key drivers.

- Hyperparameter Tuning
Apply Grid Search or Optuna to optimize tree depth, learning rate, regularization, and more.

- Feature Engineering
Enhance signal by adding lab ratios, trends, and domain-informed interactions (e.g., age × blood pressure).

- Improved Text Embeddings (for Hebrew)
Instead of TF-IDF, experiment with multilingual embeddings or Hebrew-specific models such as AlephBERT for richer clinical text representation.

- Probability Calibration & Threshold Tuning
Calibrate output scores and fine-tune classification thresholds for better prioritization.

- Add More Positive Samples
Expanding the dataset with more Y=1 cases can improve model learning and performance in imbalanced settings.

- Diagnosis Source Analysis
Use match_*_after fields to evaluate how the model performs across different diagnosis types and explore multi-label classification.

### Budget-Constrained Evaluation
To simulate a fixed testing budget, I ranked patients by their predicted risk (from the XGBoost model) and measured model performance at different Top-K thresholds — representing the number of patients that could be tested.

For example, assuming a budget to test only the top 100 patients (out of 2000 in the test set), the model achieves:

- Precision@100 = 61% (i.e., 61 out of 100 were true cases)

- Recall@100 = 70.9% (i.e., 70.9% of all true cases in the test set were captured)

This provides a practical framework to select patients for testing when resources are limited, ensuring high-risk patients are prioritized.

In [ ]:
# Re-define Precision@K and Recall@K values from earlier results
top_k = [50, 100, 200, 300, 500]
precision_at_k = [0.88, 0.61, 0.385, 0.263, 0.168]
recall_at_k = [0.5116, 0.7093, 0.8953, 0.9186, 0.9767]

# Assume test set has same distribution: 86 positives out of 1999 total
baseline_pos_rate = 86 / 1999

# Plot Recall@K and Precision@K vs Top-K
plt.figure(figsize=(10, 6))
plt.plot(top_k, precision_at_k, marker='o', label='Precision@K')
plt.plot(top_k, recall_at_k, marker='o', label='Recall@K')
plt.axhline(y=baseline_pos_rate, color='gray', linestyle='--', label='Baseline Pos Rate (Random Selection)')
plt.xlabel("Top-K Patients (Testing Budget)")
plt.ylabel("Score")
plt.title("Budget-Constrained Evaluation: Precision@K and Recall@K")
plt.xticks(top_k)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

### Budget-Constrained Evaluation plot showing how well the model performs under different screening budgets:

Interpretation:
Precision@K drops as K increases, more patients tested = more false positives.

Recall@K improves as K increases — testing more patients captures more true cases.

The gray line represents the baseline positive rate (~4.3%) you'd get from random selection.

This clearly shows the value of the model:
→ At K = 100, the model captures ~70% of all true cases with precision 14× better than random.

### Interpretation & Recommendations
 
Feature Interpretation:
The XGBoost model identifies several key predictors of hypertensive disorders:

- Blood Pressure Features:
Systolic and diastolic values — including trends and maximum readings — were among the most influential signals, suggesting early signs of elevated risk.

- Diagnosis History (4–24 months):
The number of recent diagnoses and how recent they were (recency score) strongly influenced predictions. Patients with multiple or recent events had higher risk scores.

- Lab Values:
Hemoglobin levels and ratios such as neutrophil-to-lymphocyte showed strong correlations with adverse outcomes.

- Demographics & Textual Features:
Age, capitation coefficient, and structured text-based features contributed additional signals.

The chart below shows the Top 20 most important features based on XGBoost gain:

In [ ]:
importance_df = pd.DataFrame({
    "feature": X.columns,
    "importance": xgb_model.feature_importances_
}).sort_values(by="importance", ascending=False).head(20)

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(data=importance_df, x="importance", y="feature", palette="viridis")
plt.title("Top 20 Feature Importances (XGBoost)")
plt.xlabel("Gain Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

### Screening Recommendations
With limited testing resources, the model helps prioritize high-risk patients.

- Top 100 patients: captures ~70% of true cases (Recall@100), with 61% precision.

- Top 200 patients: raises coverage to ~90%.

Thresholds can be adjusted to fit available budget, supporting efficient, targeted screening.